## KL Divergence Score

### 1 Take the simple sentences and compute the K-L divergence scores for them, in both directions. Now create a third story that is very different to the other two, add it to the program and report how its score changes relative to the first two. Explain what role epsilon and gamma play in the computation of K-L.

In [23]:
# Manos Tsagkias' program for computing Kullback-Liebler Divergence
# Using the Migge (2003) smoothening backoff
# see http://staff.science.uva.nl/~tsagias/?s=kullback
# updated for Python3 by Mark Keane 30-June-2014

import re, math, collections
from collections import defaultdict, deque

def tokenize(_str):

    stopwords = ['and', 'for', 'if', 'too', 'as', 'the', 'then', 'be', 'is', 'are', 'will', 'in', 'it', 'to', 'that', '-']
    tokens = collections.defaultdict(int)
    for m in re.finditer(r"(\w+)", _str, re.UNICODE):
        m = m.group(1).lower()
        if len(m) < 2: continue
        if m in stopwords: continue
        tokens[m] += 1
    return tokens
#end of tokenize

def kldiv(_s, _t):
    if (len(_s) == 0):
        return 1e33
    if (len(_t) == 0):
        return 1e33
    ssum = 0. + sum(_s.values())
    slen = len(_s)
    
    tsum = 0. + sum(_t.values())
    tlen = len(_t)
    vocabdiff = set(_s.keys()).difference(set(_t.keys()))
    #print(vocabdiff)
    lenvocabdiff = len(vocabdiff)
    
    #print("_s: %s" % _s)
    #print("_t: %s" % _t)
    #print("%s" % vocabdiff)

    """ epsilon """
    epsilon = min(min(_s.values())/ssum, min(_t.values())/tsum) * 0.001
    #print('epsilon: ',epsilon)
    
    """ gamma """
    gamma = 1 - lenvocabdiff * epsilon
    #print('gamma:',gamma)
    
    """ Check if distribution probabilities sum to 1"""
    sc = sum([v/ssum for v in _s.values()])  
    st = sum([v/tsum for v in _t.values()]) 
    
    if sc < 9e-6:
        print("Sum P: %e, Sum Q: %e" % (sc, st))
        print("*** ERROR: sc does not sum up to 1. Bailing out ..")
        sys.exit(2)
    if st < 9e-6:
        print("Sum P: %e, Sum Q: %e" % (sc, st))
        print("*** ERROR: st does not sum up to 1. Bailing out ..")
        sys .exit(2)

    div = 0.
    for t, v in _s.items(): 
        pts = v / ssum
        ptt = epsilon
        if t in _t:
            ptt = gamma * (_t[t] / tsum)
            
        ckl = (pts - ptt) * math.log(pts / ptt)

        div +=  ckl
    return div

#end of kldiv

d1 = """john fell down harry fell as-well down stream sun shone before down marry fine"""

d2 = """bill fell down jeff fell too down river sun shone until down belinda ill"""

d3 = """It is expensive and the battery does not last as long as I would like but that has not stopped me from thoroughly enjoying my time with the Pixel 4 XL"""


print('d1 = ',d1)
print('d2 = ',d2)
print('d3 = ',d3)


print('\n\nd1 after tokenizing:  ',tokenize(d1))
print('\nd2 after tokenizing:  ',tokenize(d2))
print('\nd3 after tokenizing:  ',tokenize(d3))



print("\n\nKL-divergence between d1 and d2:", kldiv(tokenize(d1), tokenize(d2)))
print("KL-divergence between d2 and d1:", kldiv(tokenize(d2), tokenize(d1)))
print("KL-divergence between d1 and d3:", kldiv(tokenize(d1), tokenize(d3)))
print("KL-divergence between d2 and d3:", kldiv(tokenize(d2), tokenize(d3)))

d1 =  john fell down harry fell as-well down stream sun shone before down marry fine
d2 =  bill fell down jeff fell too down river sun shone until down belinda ill
d3 =  It is expensive and the battery does not last as long as I would like but that has not stopped me from thoroughly enjoying my time with the Pixel 4 XL


d1 after tokenizing:   defaultdict(<class 'int'>, {'john': 1, 'fell': 2, 'down': 3, 'harry': 1, 'well': 1, 'stream': 1, 'sun': 1, 'shone': 1, 'before': 1, 'marry': 1, 'fine': 1})

d2 after tokenizing:   defaultdict(<class 'int'>, {'bill': 1, 'fell': 2, 'down': 3, 'jeff': 1, 'river': 1, 'sun': 1, 'shone': 1, 'until': 1, 'belinda': 1, 'ill': 1})

d3 after tokenizing:   defaultdict(<class 'int'>, {'expensive': 1, 'battery': 1, 'does': 1, 'not': 2, 'last': 1, 'long': 1, 'would': 1, 'like': 1, 'but': 1, 'has': 1, 'stopped': 1, 'me': 1, 'from': 1, 'thoroughly': 1, 'enjoying': 1, 'my': 1, 'time': 1, 'with': 1, 'pixel': 1, 'xl': 1})


KL-divergence between d1 and d2: 3.4532350